In [1]:
import numpy as np
import pandas as pd
import time
import os
import datetime as dt
import xgboost as xgb
import lightgbm as lgbm
import torch
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, mean_squared_error
import joblib as jl
import asyncio
import catboost
import matplotlib.pyplot as plt

In [54]:
# Смотрим на список collected csv-файлов
path0 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data'
path1 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\Before 2021-04-20'
path2 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\ABNB 2021-04-20'
path3 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\ABNB before 2021-04-20'
path4 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\ABNB'
path5 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\ABNB 2021-05-10'
path6 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\MRNA 2021-05-10'
path7 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\SPCE 2021-05-10'
path8 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\2021-05-10'
path9 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\2021-05-11'
path10 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\2021-05-12'
path11 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\2021-05-13'
path12 = 'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Data\\2021-05-10 - 2021-05-13'
is_csv = [x.endswith('.csv') for x in os.listdir(path12)]
np.array(os.listdir(path12))[is_csv]

array(['df_X_ABNB 2021-05-10 3 sec.csv', 'df_X_ABNB 2021-05-11 3 sec.csv',
       'df_X_ABNB 2021-05-12 1 3 sec.csv',
       'df_X_ABNB 2021-05-12 2 3 sec.csv',
       'df_X_ABNB 2021-05-13 3 sec.csv', 'df_X_MRNA 2021-05-10 3 sec.csv',
       'df_X_MRNA 2021-05-11 3 sec.csv', 'df_X_MRNA 2021-05-12 3 sec.csv',
       'df_X_MRNA 2021-05-13 3 sec.csv', 'df_X_SPCE 2021-05-10 3 sec.csv',
       'df_X_SPCE 2021-05-11 3 sec.csv', 'df_X_SPCE 2021-05-12 3 sec.csv',
       'df_X_SPCE 2021-05-13 3 sec.csv'], dtype='<U32')

In [55]:
# Списки имен df и самих df
dfs = []
dfs_list = []
for file_name in os.listdir(path12):
    if file_name.endswith('.csv'):
        var_name = file_name.split('.')[0].replace(' ','_').replace('-','_')
        dfs.append(var_name)
        globals()[var_name] = pd.read_csv(path12 + '\\' + file_name)     
        dfs_list.append(globals()[var_name])

In [56]:
# Нормальный временной столбец для всех df
for name in dfs:
    date_strs = []
    cur_df = globals()[name]
    #print(cur_df)
    if str(cur_df.iloc[0,0]).endswith(')'):
        date_lists = [x.replace('(','').replace(')','').replace(' ','').split(',') for x in cur_df.iloc[:,0]]
        for date_list in date_lists:
            date_strs.append(date_list[3]+':'+date_list[4]+':'+date_list[5]+' '+date_list[2]+'.'+date_list[1]+'.'+date_list[0])
        globals()[name].iloc[:,0] = date_strs
        globals()[name].iloc[:,0] = pd.to_datetime(globals()[name].iloc[:,0])
    else:
        globals()[name].iloc[:,0] = pd.to_datetime(globals()[name].iloc[:,0])

In [57]:
# Делаем столбец с интервалом и индикатор "хорошей" строки
for name in dfs:
    df = globals()[name]
    if '3_sec' not in name:
        df.rename(columns={'Unnamed: 0': 'Timestamp'},inplace=True)
        df['Next_timestamp'] = list(df.iloc[1:,0])+[pd.to_datetime('2000-01-01')]
        df['Timedelta'] = df['Next_timestamp'] - df['Timestamp']
        df['NNext_timestamp'] = list(df.iloc[2:,0])+[pd.to_datetime('2000-01-01'),pd.to_datetime('2000-01-01')]
        df['TTimedelta'] = df['NNext_timestamp'] - df['Next_timestamp']
        df['NNNext_timestamp'] = list(df.iloc[3:,0])+\
            [pd.to_datetime('2000-01-01'),pd.to_datetime('2000-01-01'),pd.to_datetime('2000-01-01')]
        df['TTTimedelta'] = df['NNNext_timestamp'] - df['NNext_timestamp']

        df['Good?'] = ((df['Timedelta']>=dt.timedelta(seconds=59)) & (df['Timedelta']<=dt.timedelta(minutes=1,seconds=1)) &
         (df['TTimedelta']>=dt.timedelta(seconds=59)) & (df['TTimedelta']<=dt.timedelta(minutes=1,seconds=1)) &
         (df['TTTimedelta']>=dt.timedelta(seconds=59)) & (df['TTTimedelta']<=dt.timedelta(minutes=1,seconds=1))
        )
    else:
        df.rename(columns={'Unnamed: 0': 'Timestamp'},inplace=True)
        df['Next_timestamp'] = list(df.iloc[20:,0])+[pd.to_datetime('2000-01-01')]*20
        df['Timedelta'] = df['Next_timestamp'] - df['Timestamp']
        df['NNext_timestamp'] = list(df.iloc[40:,0])+[pd.to_datetime('2000-01-01')]*40
        df['TTimedelta'] = df['NNext_timestamp'] - df['Next_timestamp']
        df['NNNext_timestamp'] = list(df.iloc[60:,0])+[pd.to_datetime('2000-01-01')]*60
        df['TTTimedelta'] = df['NNNext_timestamp'] - df['NNext_timestamp']

        df['Good?'] = ((df['Timedelta']>=dt.timedelta(seconds=59)) & (df['Timedelta']<=dt.timedelta(minutes=1,seconds=1)) &
         (df['TTimedelta']>=dt.timedelta(seconds=59)) & (df['TTimedelta']<=dt.timedelta(minutes=1,seconds=1)) &
         (df['TTTimedelta']>=dt.timedelta(seconds=59)) & (df['TTTimedelta']<=dt.timedelta(minutes=1,seconds=1))
        )

In [58]:
# Список номеров столбцов
price_cols = [1+41*k for k in range(20)]
orderbook_cols = [x for x in list(range(1,821)) if x not in price_cols]

In [59]:
# Делаем список из массивов numpy
nps_list = []
for df in dfs_list:
    nps_list.append(np.array(df))
# Добавляем в каждый массив данные для будущего вектора Y
i=0
for arr in nps_list:
    if '3_sec' not in dfs[i]:
        arr = np.hstack([arr,np.vstack([arr[1:,price_cols],np.zeros((1,len(price_cols)))])])
        arr = np.hstack([arr,np.vstack([arr[2:,price_cols],np.zeros((2,len(price_cols)))])])
        arr = np.hstack([arr,np.vstack([arr[3:,price_cols],np.zeros((3,len(price_cols)))])])
    else:
        arr = np.hstack([arr,np.vstack([arr[20:,price_cols],np.zeros((20,len(price_cols)))])])
        arr = np.hstack([arr,np.vstack([arr[40:,price_cols],np.zeros((40,len(price_cols)))])])
        arr = np.hstack([arr,np.vstack([arr[60:,price_cols],np.zeros((60,len(price_cols)))])])
    nps_list[i] = arr
    i+=1

In [9]:
# Либо это
big_arr = np.vstack(nps_list)
data = big_arr[np.array(big_arr[:,827],dtype=bool)]
data_grouped = np.apply_along_axis(
        lambda y: np.apply_along_axis(lambda x: np.sum(x),1,y[np.hsplit(np.array(orderbook_cols),200)]),1,data
        )
data_grouped = np.hstack([data_grouped,np.apply_along_axis(lambda x: x[price_cols+list(range(828,888))],1,data)])

In [60]:
# Либо это
big_arr = np.vstack(nps_list)
data = big_arr[np.array(big_arr[:,827],dtype=bool)]

part_split_cols = np.array([3,6,9,12,15,18,20])
orderbook_split_cols = [part_split_cols]
for i in list(range(39)):
    part_split_cols = part_split_cols+20
    orderbook_split_cols.append(list(part_split_cols))
orderbook_split_cols = np.array(orderbook_split_cols).ravel()
orderbook_split_cols = np.split(orderbook_cols,orderbook_split_cols)[:-1]
orderbook_split_cols_ = []
for i in orderbook_split_cols:
    if len(i)==3:
        orderbook_split_cols_.append(i)
    if len(i)==2:
        orderbook_split_cols_.append(np.hstack([i,np.array([data.shape[1]])]))

data_grouped = np.apply_along_axis(
        lambda y: np.apply_along_axis(lambda x: np.sum(x),1,y[orderbook_split_cols_]),1,np.hstack([data,np.zeros((data.shape[0],1))])
        )
data_grouped = np.hstack([data_grouped,np.apply_along_axis(lambda x: x[price_cols+list(range(828,888))],1,data)])

In [61]:
new_price_cols = list(range(data_grouped.shape[1]-80,data_grouped.shape[1]-60))
new_orderbook_cols = list(range(data_grouped.shape[1]-80))

In [10]:
# Функция нормализации массива
def normalize(arr_orig,price_cols,orderbook_cols):
    arr = arr_orig.copy()
    arr[:,price_cols+list(range(arr.shape[1]-60,arr.shape[1]))] =\
    np.apply_along_axis(lambda x: x[price_cols+list(range(arr.shape[1]-60,arr.shape[1]))]/x[price_cols[-1]],1,arr)
    arr[:,orderbook_cols] =\
    np.apply_along_axis(lambda x: x[orderbook_cols]/np.average(np.array(x[orderbook_cols],dtype=np.float64)),1,arr)
    return arr

In [62]:
# Подготавливаем окончательный X 
#big_arr = np.vstack(nps_list)
#big_arr = np.delete(big_arr,8312,0)
#big_arr_normalized = normalize(big_arr)
#data = big_arr[np.array(big_arr[:,827],dtype=bool)]
data_grouped_normalized = normalize(data_grouped,new_price_cols,new_orderbook_cols)
X = data_grouped_normalized[:,list(range(data_grouped.shape[1]-60))]

In [12]:
# Функция подготавливающая y для классификатора
def y_to_classes(y):
    if y:
        y_class=1
    else:
        y_class=0
    return int(y_class)

In [65]:
# Несколько вариантов вектора y (целевой переменной)
# Вектор - Разница максимальной цены за 3 минуты с последней ценой
y = np.apply_along_axis(lambda x: np.max(x)-1,1,np.array(data_grouped_normalized[:,list(range(data_grouped.shape[1]-60,data_grouped.shape[1]-40))]))
# Вектор - Разница минимальной цены за 3 минуты с последней ценой
y_min = np.apply_along_axis(lambda x: np.min(x)-1,1,np.array(data_grouped_normalized[:,list(range(data_grouped.shape[1]-60,data_grouped.shape[1]-40))]))
# Вектор для Классификатора
y_bool1 = (y>0.004)
y_class1 = np.vectorize(y_to_classes)(y_bool1)

y_bool2 = (y>0.005)
y_class2 = np.vectorize(y_to_classes)(y_bool2)

y_bool3 = (y>0.006)
y_class3 = np.vectorize(y_to_classes)(y_bool3)

y_bool_min0 = (y_min>-0.0015)
y_min_class0 = np.vectorize(y_to_classes)(y_bool_min0)

y_bool_min1 = (y_min>-0.002)
y_min_class1 = np.vectorize(y_to_classes)(y_bool_min1)

y_bool_min2 = (y_min>-0.003)
y_min_class2 = np.vectorize(y_to_classes)(y_bool_min2)

y_bool_max_min1 = (y>0.004) & (y_min>-0.0015)
y_max_min_class1 = np.vectorize(y_to_classes)(y_bool_max_min1)

y_bool_max_min2 = (y>0.005) & (y_min>-0.0015)
y_max_min_class2 = np.vectorize(y_to_classes)(y_bool_max_min2)

In [16]:
X_train1,X_test1,y_max_train1,y_max_test1 = train_test_split(X,y,test_size=0.25,shuffle=False)
X_train1,X_test1,y_min_train1,y_min_test1 = train_test_split(X,y_min,test_size=0.25,shuffle=False)
X_train2,X_test2,y_max_train2,y_max_test2 = train_test_split(X,y,test_size=0.2,shuffle=False)
X_train2,X_test2,y_min_train2,y_min_test2 = train_test_split(X,y_min,test_size=0.2,shuffle=False)
X_train3,X_test3,y_max_train3,y_max_test3 = train_test_split(X,y,test_size=0.15,shuffle=False)
X_train3,X_test3,y_min_train3,y_min_test3 = train_test_split(X,y_min,test_size=0.15,shuffle=False)

In [48]:
X_train = X
# 30 sec
y_train1 = y_class1
y_train2 = y_class2
y_train3 = y_class3
y_min_train0 = y_min_class0
y_min_train1 = y_min_class1
y_min_train2 = y_min_class2
y_max_min_train1 = y_max_min_class1
y_max_min_train2 = y_max_min_class2
y_max_train = y
y_min_train = y_min

In [51]:
# 1 min
y_train1_ = y_class1
y_train2_ = y_class2
y_train3_ = y_class3
y_min_train0_ = y_min_class0
y_min_train1_ = y_min_class1
y_min_train2_ = y_min_class2
y_max_min_train1_ = y_max_min_class1
y_max_min_train2_ = y_max_min_class2
y_max_train_ = y
y_min_train_ = y_min

In [64]:
X_test = X
# 30 sec
y_test1 = y_class1
y_test2 = y_class2
y_test3 = y_class3
y_min_test0 = y_min_class0
y_min_test1 = y_min_class1
y_min_test2 = y_min_class2
y_max_min_test1 = y_max_min_class1
y_max_min_test2 = y_max_min_class2
y_max_test = y
y_min_test = y_min

In [66]:
# 1 min
y_test1_ = y_class1
y_test2_ = y_class2
y_test3_ = y_class3
y_min_test0_ = y_min_class0
y_min_test1_ = y_min_class1
y_min_test2_ = y_min_class2
y_max_min_test1_ = y_max_min_class1
y_max_min_test2_ = y_max_min_class2
y_max_test_ = y
y_min_test_ = y_min

In [67]:
del X, y, dfs_list, nps_list, big_arr, data, data_grouped, data_grouped_normalized

Данные готовы. Переходим к обучению

### Классификация!

Unbalanced data

In [16]:
def display_summary(true,pred):
    tn, fp, fn, tp = confusion_matrix(true,pred).ravel()
    print('confusion matrix')
    print(np.array([[tp,fp],[fn,tn]]))
    print('sensitivity is %f',1.*tp/(tp+fn))
    print('specificity is %f',1.*tn/(tn+fp))
    print('accuracy is %f',1.*(tp+tn)/(tp+tn+fp+fn))
    print('balanced accuracy is %',1./2*(1.*tp/(tp+fn)+1.*tn/(tn+fp)))

In [122]:
ratio = len(y_train2[y_train2==0])/len(y_train2[y_train2==1])

In [129]:
ratio_min = len(y_train2[y_min_train1==0])/len(y_train2[y_min_train1==1])

In [130]:
ratio,ratio_min

(39.633948339483396, 0.23818983223639825)

In [175]:
jl.dump(xgbc_25,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\xgbc_25.jl')
jl.dump(xgbc_375,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\xgbc_375.jl')
jl.dump(xgbc_5,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\xgbc_5.jl')
jl.dump(xgbc_625,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\xgbc_625.jl')
jl.dump(xgbc_75,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\xgbc_75.jl')

XGBClassifier(objective='reg:logistic', scale_pos_weight=0.25)

In [160]:
xgbc_ratios = [0.25,0.375,0.5,0.625,0.75,0.875]
conf_matrices = {}
max_preds = []
for ratio in xgbc_ratios[::-1]:
    globals()['xgbc_' + str(ratio).split('.')[-1]] = xgb.XGBClassifier(objective='reg:logistic',scale_pos_weight=ratio)
    globals()['xgbc_' + str(ratio).split('.')[-1]].fit(X_train,y_train1)
    globals()['y_xgbc_' + str(ratio).split('.')[-1]] = globals()['xgbc_' + str(ratio).split('.')[-1]].predict(X_test)
    max_preds.append(globals()['y_xgbc_' + str(ratio).split('.')[-1]])
    tn, fp, fn, tp = confusion_matrix(y_test1,np.int16(np.round(globals()['y_xgbc_' + str(ratio).split('.')[-1]]))).ravel()
    conf_matrices[ratio] = [[tp,fp],[fn,tn]]
    print('_______________')
    print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))
    print('max_avg: ' + str(np.round(np.average(y_max_test[globals()['y_xgbc_' + str(ratio).split('.')[-1]]==1]),4)))
    print('min_avg: ' + str(np.round(np.average(y_min_test[globals()['y_xgbc_' + str(ratio).split('.')[-1]]==1]),4)))

_______________
for ratio 875 tp/(tp+fp) is: 0.647
max_avg: 0.0077
min_avg: -0.007
_______________
for ratio 75 tp/(tp+fp) is: 0.727
max_avg: 0.008
min_avg: -0.007
_______________
for ratio 625 tp/(tp+fp) is: 0.769
max_avg: 0.01
min_avg: -0.0043
_______________
for ratio 5 tp/(tp+fp) is: 1.0
max_avg: 0.0088
min_avg: -0.0048
_______________
for ratio 375 tp/(tp+fp) is: 1.0
max_avg: 0.01
min_avg: -0.0052
_______________
for ratio 25 tp/(tp+fp) is: 1.0
max_avg: 0.0087
min_avg: -0.0042


In [46]:
xgbc_ratios = [0.75,0.5,0.25]
conf_matrices = {}
max_preds = []
for ratio in xgbc_ratios[::-1]:
    globals()['xgbc_min_' + str(ratio).split('.')[-1]] = xgb.XGBClassifier(objective='reg:logistic',scale_pos_weight=ratio)
    globals()['xgbc_min_' + str(ratio).split('.')[-1]].fit(X_train,y_min_train0)
    globals()['y_xgbc_min_' + str(ratio).split('.')[-1]] = globals()['xgbc_min_' + str(ratio).split('.')[-1]].predict(X_test)
    max_preds.append(globals()['y_xgbc_min_' + str(ratio).split('.')[-1]])
    tn, fp, fn, tp = confusion_matrix(y_min_test0,np.int16(np.round(globals()['y_xgbc_min_' + str(ratio).split('.')[-1]]))).ravel()
    conf_matrices[ratio] = [[tp,fp],[fn,tn]]
    print('_______________')
    print('confusion matrix: '+ str(conf_matrices[ratio]))
    print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))
    print('max_avg: ' + str(np.round(np.average(y_max_test[globals()['y_xgbc_min_' + str(ratio).split('.')[-1]]==1]),4)))
    print('min_avg: ' + str(np.round(np.average(y_min_test[globals()['y_xgbc_min_' + str(ratio).split('.')[-1]]==1]),4)))

_______________
confusion matrix: [[9477, 2599], [20497, 19987]]
for ratio 25 tp/(tp+fp) is: 0.785
max_avg: 0.001
min_avg: -0.0011
_______________
confusion matrix: [[21188, 10396], [8786, 12190]]
for ratio 5 tp/(tp+fp) is: 0.671
max_avg: 0.0013
min_avg: -0.0015
_______________
confusion matrix: [[26052, 15924], [3922, 6662]]
for ratio 75 tp/(tp+fp) is: 0.621
max_avg: 0.0015
min_avg: -0.0016


In [32]:
xgbc_5 = jl.load('D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\xgbc_5.jl')
xgbc_625 = jl.load('D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\xgbc_625.jl')
xgbc_75 = jl.load('D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\xgbc_75.jl')

In [34]:
y_xgbc_625 = xgbc_625.predict(X_test)
y_xgbc_5 = xgbc_5.predict(X_test)
y_xgbc_75 = xgbc_75.predict(X_test)

In [64]:
# Результаты системы с двумя классификаторами
print(np.round(np.average(y_max_test[(y_xgbc_75==1)&(y_xgbc_min_5==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_75==1)&(y_xgbc_min_5==1)]),4),\
        len(y_max_test[(y_xgbc_75==1)&(y_xgbc_min_5==1)]))
print(np.round(np.average(y_max_test[(y_xgbc_625==1)&(y_xgbc_min_5==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_625==1)&(y_xgbc_min_5==1)]),4),\
        len(y_max_test[(y_xgbc_625==1)&(y_xgbc_min_5==1)]))
print(np.round(np.average(y_max_test[(y_xgbc_5==1)&(y_xgbc_min_5==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_5==1)&(y_xgbc_min_5==1)]),4),\
        len(y_max_test[(y_xgbc_5==1)&(y_xgbc_min_5==1)]))
print(np.round(np.average(y_max_test[(y_xgbc_75==1)&(y_xgbc_min_25==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_75==1)&(y_xgbc_min_25==1)]),4),\
        len(y_max_test[(y_xgbc_75==1)&(y_xgbc_min_25==1)]))
print(np.round(np.average(y_max_test[(y_xgbc_625==1)&(y_xgbc_min_25==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_625==1)&(y_xgbc_min_25==1)]),4),\
        len(y_max_test[(y_xgbc_625==1)&(y_xgbc_min_25==1)]))
print(np.round(np.average(y_max_test[(y_xgbc_5==1)&(y_xgbc_min_25==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_5==1)&(y_xgbc_min_25==1)]),4),\
        len(y_max_test[(y_xgbc_5==1)&(y_xgbc_min_25==1)]))

0.0061 0.0 1
nan nan 0
nan nan 0
nan nan 0
nan nan 0
nan nan 0


In [73]:
# Тренируем систему с одним классификатором
xgbc_ratios = [0.75,0.625,0.5]
conf_matrices = {}
max_preds = []
for ratio in xgbc_ratios[::-1]:
    globals()['xgbc_max_min_' + str(ratio).split('.')[-1]] = xgb.XGBClassifier(objective='reg:logistic',scale_pos_weight=ratio)
    globals()['xgbc_max_min_' + str(ratio).split('.')[-1]].fit(X_train,y_max_min_train1)
    globals()['y_xgbc_max_min_' + str(ratio).split('.')[-1]] = globals()['xgbc_max_min_' + str(ratio).split('.')[-1]].predict(X_test)
    max_preds.append(globals()['y_xgbc_max_min_' + str(ratio).split('.')[-1]])
    tn, fp, fn, tp = confusion_matrix(y_max_min_test1,np.int16(np.round(globals()['y_xgbc_max_min_' + str(ratio).split('.')[-1]]))).ravel()
    conf_matrices[ratio] = [[tp,fp],[fn,tn]]
    print('_______________')
    print('confusion matrix: '+ str(conf_matrices[ratio]))
    print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))
    print('max_avg: ' + str(np.round(np.average(y_max_test[globals()['y_xgbc_max_min_' + str(ratio).split('.')[-1]]==1]),4)))
    print('min_avg: ' + str(np.round(np.average(y_min_test[globals()['y_xgbc_max_min_' + str(ratio).split('.')[-1]]==1]),4)))

_______________
confusion matrix: [[0, 0], [5041, 47519]]
for ratio 5 tp/(tp+fp) is: nan
max_avg: nan
min_avg: nan


<ipython-input-73-7a6ef545f22d>:14: RuntimeWarning: invalid value encountered in longlong_scalars
  print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))


_______________
confusion matrix: [[0, 2], [5041, 47517]]
for ratio 625 tp/(tp+fp) is: 0.0
max_avg: 0.0121
min_avg: -0.0109
_______________
confusion matrix: [[1, 1], [5040, 47518]]
for ratio 75 tp/(tp+fp) is: 0.5
max_avg: 0.0094
min_avg: -0.0069


In [74]:
print(np.round(np.average(y_max_test[(y_xgbc_max_min_75==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_max_min_75==1)]),4),\
        len(y_max_test[(y_xgbc_max_min_75==1)]))
print(np.round(np.average(y_max_test[(y_xgbc_max_min_625==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_max_min_625==1)]),4),\
        len(y_max_test[(y_xgbc_max_min_625==1)]))
print(np.round(np.average(y_max_test[(y_xgbc_max_min_5==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_max_min_5==1)]),4),\
        len(y_max_test[(y_xgbc_max_min_5==1)]))

0.0094 -0.0069 2
0.0121 -0.0109 2
nan nan 0


In [69]:
cbc_625 = catboost.CatBoostClassifier(class_weights=[1,0.625],verbose=False,eval_metric='BalancedAccuracy')
cbc_5 = catboost.CatBoostClassifier(class_weights=[1,0.5],verbose=False,eval_metric='BalancedAccuracy')
cbc_75 = catboost.CatBoostClassifier(class_weights=[1,0.75],verbose=False,eval_metric='BalancedAccuracy')
cbc_375 = catboost.CatBoostClassifier(class_weights=[1,0.375],verbose=False,eval_metric='BalancedAccuracy')

In [85]:
cbc_025 = catboost.CatBoostClassifier(class_weights=[1,0.025],verbose=False,eval_metric='BalancedAccuracy')
start_time = time.time()
cbc_025.fit(X_train,y_train2)
print(np.round(time.time() - start_time))
y_cbc_025 = cbc_025.predict(X_test)
display_summary(y_test2,np.int16(np.round(y_cbc_025)))
print(np.round(np.average(y_max_test[(y_cbc_025==1)]),4),\
        np.round(np.average(y_min_test[(y_cbc_025==1)]),4),\
        len(y_max_test[(y_cbc_025==1)]))

256.0
confusion matrix
[[   27    11]
 [ 4166 48356]]
sensitivity is %f 0.006439303601240162
specificity is %f 0.9997725722083238
accuracy is %f 0.9205289193302892
balanced accuracy is % 0.503105937904782
0.0075 -0.0048 38


In [86]:
jl.dump(cbc_35,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\cbc_35.jl')
jl.dump(cbc_2,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\cbc_2.jl')
jl.dump(cbc_1,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\cbc_1.jl')
jl.dump(cbc_05,'D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\cbc_05.jl')

['D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\cbc_05.jl']

In [87]:
cbc_2 = catboost.CatBoostClassifier(class_weights=[1,0.2],verbose=False,eval_metric='BalancedAccuracy')
start_time = time.time()
cbc_2.fit(X_train,y_max_min_train2)
print(np.round(time.time() - start_time))
y_cbc_2 = cbc_2.predict(X_test)
display_summary(y_max_min_test2,np.int16(np.round(y_cbc_2)))
print(np.round(np.average(y_max_test[(y_cbc_2==1)]),4),\
        np.round(np.average(y_min_test[(y_cbc_2==1)]),4),\
        len(y_max_test[(y_cbc_2==1)]))

cbc_1 = catboost.CatBoostClassifier(class_weights=[1,0.1],verbose=False,eval_metric='BalancedAccuracy')
start_time = time.time()
cbc_1.fit(X_train,y_max_min_train2)
print(np.round(time.time() - start_time))
y_cbc_1 = cbc_1.predict(X_test)
display_summary(y_max_min_test2,np.int16(np.round(y_cbc_1)))
print(np.round(np.average(y_max_test[(y_cbc_1==1)]),4),\
        np.round(np.average(y_min_test[(y_cbc_1==1)]),4),\
        len(y_max_test[(y_cbc_1==1)]))

cbc_05 = catboost.CatBoostClassifier(class_weights=[1,0.05],verbose=False,eval_metric='BalancedAccuracy')
start_time = time.time()
cbc_05.fit(X_train,y_max_min_train2)
print(np.round(time.time() - start_time))
y_cbc_05 = cbc_05.predict(X_test)
display_summary(y_max_min_test2,np.int16(np.round(y_cbc_05)))
print(np.round(np.average(y_max_test[(y_cbc_05==1)]),4),\
        np.round(np.average(y_min_test[(y_cbc_05==1)]),4),\
        len(y_max_test[(y_cbc_05==1)]))

297.0
confusion matrix
[[    2     8]
 [ 3362 49188]]
sensitivity is %f 0.0005945303210463733
specificity is %f 0.9998373851532645
accuracy is %f 0.935882800608828
balanced accuracy is % 0.5002159577371554
0.0094 -0.0067 10
297.0
confusion matrix
[[    0     6]
 [ 3364 49190]]
sensitivity is %f 0.0
specificity is %f 0.9998780388649484
accuracy is %f 0.935882800608828
balanced accuracy is % 0.4999390194324742
0.0093 -0.0093 6


KeyboardInterrupt: 

In [88]:
cbc_35 = catboost.CatBoostClassifier(class_weights=[1,0.35],verbose=False,eval_metric='BalancedAccuracy')
start_time = time.time()
cbc_35.fit(X_train,y_max_min_train2)
print(np.round(time.time() - start_time))
y_cbc_35 = cbc_35.predict(X_test)
display_summary(y_max_min_test2,np.int16(np.round(y_cbc_35)))
print(np.round(np.average(y_max_test[(y_cbc_35==1)]),4),\
        np.round(np.average(y_min_test[(y_cbc_35==1)]),4),\
        len(y_max_test[(y_cbc_35==1)]))

295.0
confusion matrix
[[    1    14]
 [ 3363 49182]]
sensitivity is %f 0.00029726516052318666
specificity is %f 0.9997154240182129
accuracy is %f 0.9357496194824962
balanced accuracy is % 0.500006344589368
0.0084 -0.0073 15


### Тесты 2021-05-14

In [33]:
# данные от 2021-05-13
y_xgbc_625 = xgbc_625.predict(X_test)
display_summary(y_test2,np.int16(np.round(y_xgbc_625)))
print(np.round(np.average(y_max_test[(y_xgbc_625==1)]),4),\
        np.round(np.average(y_min_test[(y_xgbc_625==1)]),4),\
        len(y_max_test[(y_xgbc_625==1)]))

confusion matrix
[[    7    16]
 [ 4459 24082]]
sensitivity is %f 0.001567398119122257
specificity is %f 0.9993360444850196
accuracy is %f 0.8433342669093964
balanced accuracy is % 0.5004517213020709
0.0063 -0.0181 23


In [34]:
cbc_35 = jl.load('D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\Classifiers group of 3\\cbc_35.jl')
cbc_2 = jl.load('D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\Classifiers group of 3\\cbc_2.jl')
cbc_1 = jl.load('D:\\Python Projects\\Stonks\\Tinkoff Online Trading\\Models\\Classifiers group of 3\\cbc_1.jl')

In [35]:
# данные от 2021-05-13
y_cbc_35 = cbc_35.predict(X_test)
display_summary(y_test2,np.int16(np.round(y_cbc_35)))
print(np.round(np.average(y_max_test[(y_cbc_35==1)]),4),\
        np.round(np.average(y_min_test[(y_cbc_35==1)]),4),\
        len(y_max_test[(y_cbc_35==1)]))

confusion matrix
[[   63    44]
 [ 4403 24054]]
sensitivity is %f 0.014106583072100314
specificity is %f 0.9981741223338036
accuracy is %f 0.8443145217756617
balanced accuracy is % 0.506140352702952
0.0083 -0.0138 107


In [36]:
# данные от 2021-05-13
y_cbc_2 = cbc_2.predict(X_test)
display_summary(y_test2,np.int16(np.round(y_cbc_2)))
print(np.round(np.average(y_max_test[(y_cbc_2==1)]),4),\
        np.round(np.average(y_min_test[(y_cbc_2==1)]),4),\
        len(y_max_test[(y_cbc_2==1)]))

confusion matrix
[[   56    33]
 [ 4410 24065]]
sensitivity is %f 0.012539184952978056
specificity is %f 0.9986305917503527
accuracy is %f 0.8444545581851282
balanced accuracy is % 0.5055848883516654
0.0091 -0.0146 89


In [37]:
# данные от 2021-05-13
y_cbc_1 = cbc_1.predict(X_test)
display_summary(y_test2,np.int16(np.round(y_cbc_1)))
print(np.round(np.average(y_max_test[(y_cbc_1==1)]),4),\
        np.round(np.average(y_min_test[(y_cbc_1==1)]),4),\
        len(y_max_test[(y_cbc_1==1)]))

confusion matrix
[[   41    36]
 [ 4425 24062]]
sensitivity is %f 0.009180474697716077
specificity is %f 0.9985061000912939
accuracy is %f 0.8438243943425291
balanced accuracy is % 0.503843287394505
0.0083 -0.0154 77


### Try to train with 30sec y

In [77]:
xgbc_ratios = [0.01,0.35,1]
conf_matrices = {}
max_preds = []
for ratio in xgbc_ratios[::-1]:
    globals()['xgbc_' + str(ratio).split('.')[-1]] = xgb.XGBClassifier(objective='reg:logistic',scale_pos_weight=ratio)
    globals()['xgbc_' + str(ratio).split('.')[-1]].fit(X_train,y_max_min_train1)
    globals()['y_xgbc_' + str(ratio).split('.')[-1]] = globals()['xgbc_' + str(ratio).split('.')[-1]].predict(X_test)
    max_preds.append(globals()['y_xgbc_' + str(ratio).split('.')[-1]])
    tn, fp, fn, tp = confusion_matrix(y_max_min_test1,np.int16(np.round(globals()['y_xgbc_' + str(ratio).split('.')[-1]]))).ravel()
    conf_matrices[ratio] = [[tp,fp],[fn,tn]]
    print('_______________')
    print('confusion matrix: '+ str(conf_matrices[ratio]))
    print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))
    print('max_avg: ' + str(np.round(np.average(y_max_test[globals()['y_xgbc_' + str(ratio).split('.')[-1]]==1]),4)))
    print('min_avg: ' + str(np.round(np.average(y_min_test[globals()['y_xgbc_' + str(ratio).split('.')[-1]]==1]),4)))

_______________
confusion matrix: [[6, 19], [2811, 138815]]
for ratio 1 tp/(tp+fp) is: 0.24
max_avg: 0.0021
min_avg: -0.0023
_______________
confusion matrix: [[0, 2], [2817, 138832]]
for ratio 35 tp/(tp+fp) is: 0.0
max_avg: 0.0003
min_avg: -0.003
_______________
confusion matrix: [[0, 0], [2817, 138834]]
for ratio 01 tp/(tp+fp) is: nan
max_avg: nan
min_avg: nan


<ipython-input-77-d9b9432f00a4>:13: RuntimeWarning: invalid value encountered in longlong_scalars
  print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))


In [78]:
cbc_ratios = [0.01,0.1,1]
conf_matrices = {}
max_preds = []
for ratio in cbc_ratios[::-1]:
    globals()['cbc_' + str(ratio).split('.')[-1]] = \
                        catboost.CatBoostClassifier(class_weights=[1,ratio],verbose=False,eval_metric='BalancedAccuracy')
    globals()['cbc_' + str(ratio).split('.')[-1]].fit(X_train,y_max_min_train1)
    globals()['y_cbc_' + str(ratio).split('.')[-1]] = globals()['cbc_' + str(ratio).split('.')[-1]].predict(X_test)
    max_preds.append(globals()['y_cbc_' + str(ratio).split('.')[-1]])
    tn, fp, fn, tp = confusion_matrix(y_max_min_test1,np.int16(np.round(globals()['y_cbc_' + str(ratio).split('.')[-1]]))).ravel()
    conf_matrices[ratio] = [[tp,fp],[fn,tn]]
    print('_______________')
    print('confusion matrix: '+ str(conf_matrices[ratio]))
    print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))
    print('max_avg: ' + str(np.round(np.average(y_max_test[globals()['y_cbc_' + str(ratio).split('.')[-1]]==1]),4)))
    print('min_avg: ' + str(np.round(np.average(y_min_test[globals()['y_cbc_' + str(ratio).split('.')[-1]]==1]),4)))

_______________
confusion matrix: [[10, 34], [2807, 138800]]
for ratio 1 tp/(tp+fp) is: 0.227
max_avg: 0.0032
min_avg: -0.0034
_______________
confusion matrix: [[10, 36], [2807, 138798]]
for ratio 1 tp/(tp+fp) is: 0.217
max_avg: 0.0035
min_avg: -0.0026
_______________
confusion matrix: [[0, 3], [2817, 138831]]
for ratio 01 tp/(tp+fp) is: 0.0
max_avg: 0.0014
min_avg: -0.0059


### Try to train with 1min y

In [79]:
xgbc_ratios = [0.01,0.1,1]
conf_matrices = {}
max_preds = []
for ratio in xgbc_ratios[::-1]:
    globals()['xgbc_' + str(ratio).split('.')[-1]] = xgb.XGBClassifier(objective='reg:logistic',scale_pos_weight=ratio)
    globals()['xgbc_' + str(ratio).split('.')[-1]].fit(X_train,y_max_min_train2)
    globals()['y_xgbc_' + str(ratio).split('.')[-1]] = globals()['xgbc_' + str(ratio).split('.')[-1]].predict(X_test)
    max_preds.append(globals()['y_xgbc_' + str(ratio).split('.')[-1]])
    tn, fp, fn, tp = confusion_matrix(y_max_min_test2,np.int16(np.round(globals()['y_xgbc_' + str(ratio).split('.')[-1]]))).ravel()
    conf_matrices[ratio] = [[tp,fp],[fn,tn]]
    print('_______________')
    print('confusion matrix: '+ str(conf_matrices[ratio]))
    print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))
    print('max_avg: ' + str(np.round(np.average(y_max_test[globals()['y_xgbc_' + str(ratio).split('.')[-1]]==1]),4)))
    print('min_avg: ' + str(np.round(np.average(y_min_test[globals()['y_xgbc_' + str(ratio).split('.')[-1]]==1]),4)))

_______________
confusion matrix: [[3, 23], [1625, 140000]]
for ratio 1 tp/(tp+fp) is: 0.115
max_avg: 0.0028
min_avg: -0.0049
_______________
confusion matrix: [[0, 0], [1628, 140023]]
for ratio 1 tp/(tp+fp) is: nan
max_avg: nan
min_avg: nan


<ipython-input-79-2d39ac0468bb>:13: RuntimeWarning: invalid value encountered in longlong_scalars
  print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))


_______________
confusion matrix: [[0, 0], [1628, 140023]]
for ratio 01 tp/(tp+fp) is: nan
max_avg: nan
min_avg: nan


In [80]:
cbc_ratios = [0.01,0.1,1]
conf_matrices = {}
max_preds = []
for ratio in cbc_ratios[::-1]:
    globals()['cbc_' + str(ratio).split('.')[-1]] = \
                        catboost.CatBoostClassifier(class_weights=[1,ratio],verbose=False,eval_metric='BalancedAccuracy')
    globals()['cbc_' + str(ratio).split('.')[-1]].fit(X_train,y_max_min_train2)
    globals()['y_cbc_' + str(ratio).split('.')[-1]] = globals()['cbc_' + str(ratio).split('.')[-1]].predict(X_test)
    max_preds.append(globals()['y_cbc_' + str(ratio).split('.')[-1]])
    tn, fp, fn, tp = confusion_matrix(y_max_min_test2,np.int16(np.round(globals()['y_cbc_' + str(ratio).split('.')[-1]]))).ravel()
    conf_matrices[ratio] = [[tp,fp],[fn,tn]]
    print('_______________')
    print('confusion matrix: '+ str(conf_matrices[ratio]))
    print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))
    print('max_avg: ' + str(np.round(np.average(y_max_test[globals()['y_cbc_' + str(ratio).split('.')[-1]]==1]),4)))
    print('min_avg: ' + str(np.round(np.average(y_min_test[globals()['y_cbc_' + str(ratio).split('.')[-1]]==1]),4)))

_______________
confusion matrix: [[1, 3], [1627, 140020]]
for ratio 1 tp/(tp+fp) is: 0.25
max_avg: 0.0036
min_avg: -0.0025
_______________
confusion matrix: [[1, 1], [1627, 140022]]
for ratio 1 tp/(tp+fp) is: 0.5
max_avg: 0.0087
min_avg: -0.0015
_______________
confusion matrix: [[0, 0], [1628, 140023]]
for ratio 01 tp/(tp+fp) is: nan
max_avg: nan
min_avg: nan


<ipython-input-80-b1b45637e538>:14: RuntimeWarning: invalid value encountered in longlong_scalars
  print('for ratio '+ str(ratio).split('.')[-1]+' tp/(tp+fp) is: ' + str(np.round(tp/(tp+fp),3)))
